In [1]:
from elliot.run import run_experiment
from lib.data_generation import generate_data
import zipfile
import io
import requests
import os
import pandas as pd
import numpy as np
import yaml
import copy
from lenskit import crossfold as xf


__/\\\\\\\\\\\\\\\___/\\\\\\______/\\\\\\_________________________________________        
 _\/\\\///////////___\////\\\_____\////\\\_________________________________________       
  _\/\\\_________________\/\\\________\/\\\______/\\\_____________________/\\\______      
   _\/\\\\\\\\\\\_________\/\\\________\/\\\_____\///_______/\\\\\______/\\\\\\\\\\\_     
    _\/\\\///////__________\/\\\________\/\\\______/\\\____/\\\///\\\___\////\\\////__    
     _\/\\\_________________\/\\\________\/\\\_____\/\\\___/\\\__\//\\\_____\/\\\______   
      _\/\\\_________________\/\\\________\/\\\_____\/\\\__\//\\\__/\\\______\/\\\_/\\__  
       _\/\\\\\\\\\\\\\\\___/\\\\\\\\\___/\\\\\\\\\__\/\\\___\///\\\\\/_______\//\\\\\___ 
        _\///////////////___\/////////___\/////////___\///______\/////__________\/////____
Version Number: 0.3.1


### DMF experiment

Generate train-test splits.

In [2]:
location = 'data/'

In [7]:
pd.read_csv(
    'data/fairbook/fairbook_events.csv', header=0,
)

,user,item,rating
0,276847,0,8
1,276847,1,10
2,276847,2,10
3,276847,3,10
4,276847,4,10
...,...,...,...
88547,276688,6837,7
88548,276688,1628,6
88549,276688,4464,8
88550,276688,5236,10


In [8]:
current_splits = []
data_strategy = 'fairbook'
# generate the data
ratings = pd.read_csv(
    'data/fairbook/fairbook_events.csv', header=0,
)

sample = xf.SampleFrac(0.2, rng_spec=0)
sets = [i for i in enumerate(xf.partition_users(ratings, 5, sample, rng_spec=0))]
for j, tp in sets:
    current_splits.append([tp[0], tp[1]])

    tp[0].to_csv(
        location + data_strategy + "_fold_" + str(j + 1) + "_train.csv", index=False
    )
    tp[1].to_csv(
        location + data_strategy + "_fold_" + str(j + 1) + "_test.csv", index=False
    )



Change data format.

In [9]:
for i in range(1, 6):
    tr = pd.read_csv("data/fairbook_fold_"+str(i)+"_train.csv")
    te = pd.read_csv("data/fairbook_fold_"+str(i)+"_test.csv")
    np.savetxt("data/fairbook_fold_"+str(i)+"_train.tsv", tr,delimiter='\t',fmt='%i')
    np.savetxt("data/fairbook_fold_"+str(i)+"_test.tsv", te,delimiter='\t',fmt='%i')
    

Experiment.

In [ ]:
# possible values for the hyperparameters
mlp_values = ['(64,32)', '(64,64)']
batch_size_values = [256, 512]

In [ ]:
for i in range(1, 6):
    print('Start for ', i, '!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!')
    with open('config_files/fairbook'+str(i)+'.yml', 'r') as f: # open the relevant yaml file
        base_config = yaml.safe_load(f)
    
    for mlp in mlp_values:
        for batch_size in batch_size_values:
            print("We re doing the following: ", mlp, batch_size)
            # Make a copy of the base configuration
            config = copy.deepcopy(base_config)
            # Update the configuration with the current hyperparameters
            config['experiment']['models']['DMF']['user_mlp'] = mlp
            config['experiment']['models']['DMF']['item_mlp'] = mlp
            config['experiment']['models']['DMF']['batch_size'] = batch_size

            # Write the configuration to a temporary file
            with open('config_files/temp_config.yml', 'w') as f:
                yaml.dump(config, f)

            # Run the experiment with the current configuration
            run_experiment('config_files/temp_config.yml')
            
            
            # Remove the temp file
            os.remove('config_files/temp_config.yml')    